<h1>Ford Care Guesser</h1>

Modelo desenvolvido a fim de ver se os componentes do trem de força correm risco ou não

In [9]:
import pandas as pd
pd.set_option('display.max_columns',None)

from datetime import datetime
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pickle

def get_train_data():
    df = pd.read_csv("../data/ford_guesser_fuel.csv", index_col='Unnamed: 0')
    return df

def format_guesser_data(df):
    
    df = df.join(pd.get_dummies(df.MARK))
    df = df.join(pd.get_dummies(df.MODEL))
    df.drop(['MARK','MODEL','VEHICLE_ID','ERROR CODE'],inplace=True,axis=1)
    df.AUTOMATIC = df.AUTOMATIC.replace({'s':1, 'n':0})
    df.DTC_NUMBER = df.DTC_NUMBER.replace({'MIL is OFF1 codes':1, 'MIL is OFF0 codes':0})
    df['SHORT TERM FUEL TRIM BANK 1'].fillna(df['SHORT TERM FUEL TRIM BANK 1'].mean(), inplace=True)

    for c in df.columns:
        if df[c].isna().sum() > 0:
            df[c] = df[c].fillna(df[c].mean())
    return df

def train_guesser(df, model):
    x = df[['CAR_YEAR', 'AIR_INTAKE_TEMP',
           'ENGINE_COOLANT_TEMP', 'ENGINE_LOAD', 'ENGINE_POWER', 'ENGINE_RPM',
           'INTAKE_MANIFOLD_PRESSURE', 'SPEED', 'THROTTLE_POS']]
    

    y = df['ALERT']
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
    
    model.fit(x_train, y_train)
    
    print(metrics.accuracy_score(model.predict(x_test),y_test))
                           
    return model.fit(x_train, y_train)

def export_guesser_pickle(knn):
    filename = '../app/ford_care/ford_guesser.sav'
    pickle.dump(knn, open(filename, 'wb'))
    
if __name__ == "__main__":
    df = get_train_data()
    df = format_guesser_data(df)
    knn = KNeighborsClassifier(n_neighbors=2)
    knn_trained = train_guesser(df, knn)
    export_guesser_pickle(knn_trained)

0.8005375845768838


In [ ]:
metrics.accuracy_score( knn_trained.